In [ ]:
from pathlib import Path
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder
import pickle
import bz2
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind


In [ ]:
!pip install gdown

You should consider upgrading via the '/home/dmitry/anaconda3/bin/python -m pip install --upgrade pip' command.


Download https://drive.google.com/drive/folders/1BZFez3OFs7QMzeDUz-8E0SgnRy227pnG

Unzip

!mkdir data !tar -xf inno_stats.tar.gz -C ./data

In [ ]:
!gdown --id 13Qm6ztAmVyBHvo_mch6gk-2VYKUHeRuu
!mkdir data 
!tar -xf inno_stats.tar.gz -C ./data

Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=13Qm6ztAmVyBHvo_mch6gk-2VYKUHeRuu 



In [ ]:
g_9 = pd.read_csv('./data/09_groups.csv')
g_10 = pd.read_csv('./data/10_groups.csv')
g_11 = pd.read_csv('./data/11_groups.csv')
g_9.head()

,date,id_doc,id_order,id_card,id_tov,id_kontr,quantity,sum,is_green,id_group
0,2020-09-01 17:01:25,1283228,0.0,1538855,52,271,1.0,108.00,0,10004
1,2020-09-01 17:35:36,8873113,0.0,267307,52,271,1.0,108.00,0,10004
2,2020-09-01 19:19:21,12712899,0.0,610220,52,271,1.0,107.95,0,10004
3,2020-09-01 21:53:34,21535283,0.0,441497,52,271,1.0,108.00,0,10004
4,2020-09-01 18:42:31,642341,0.0,1065358,61,97,2.0,88.00,0,10091


In [ ]:
p_g = pd.read_csv('./data/product_groups.csv', delimiter=';', encoding='windows-1251')
p_g.head()

,id_tov,id_group,name_group
0,52,10004,Кисломолочные продукты
1,75,10004,Кисломолочные продукты
2,77,10004,Кисломолочные продукты
3,143,10004,Кисломолочные продукты
4,151,10004,Кисломолочные продукты


In [ ]:
bad_ids = pd.read_csv('bad_ids.csv')
bad_ids.head()

,Unnamed: 0,id_card
0,0,806096
1,1,1438102
2,2,72344
3,3,427981
4,4,1528414


In [ ]:
df = g_9

In [ ]:
df = df[~df.id_card.isin(bad_ids.id_card.values)]
df.info()

Number of samples decreasef from 42 100 858 to 38 466 003

## Task 1

Prove that buying discounted products depends on the average order sum. (It is more probable that the discounted product will be bought if bunch of money is going to be spend)

Prove that buying discounted products depends on the average order sum. (It is more probable that the discounted product will be bought if bunch of money is going to be spend)

H0: discounted products buyid does not depend on the average order sum

HA: discounted products buyid does depend on the average order sum

In [ ]:
avg_sum = df.groupby('id_doc')['sum'].mean()
avg_disc = df.groupby('id_doc')['is_green'].count()

ttest_ind(avg_disc.values, avg_sum.values)

P-values is low, H<sub>0</sub> is rejected, 

So, discounted products DEPEND on the average order sum

## Task 2
Prove that the frequency of shopping grows around the wage taking days.

In [ ]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d %H:%M:%S')

recipts_per_day = df.groupby(df.date.dt.day)['id_doc'].count()

In [ ]:
plt.figure(figsize=(50, 30))
plt.locator_params(axis='x', nbins=31)
plt.grid()
plt.plot(recipts_per_day)

It is seen that local minimums are at days 5, 20 and 26. If we suggest that paydays are days 1-5 and 20-25 then we can see that before those days function decreases - rare buys. Also, function increases right after mentioned days -people buy more frequently. 

## Task 3
Define such cohort existance:
1. Frequent buyers - The users that are shopping frequently(daily, weekly, monthly) 
2. Average purchase sum - The customers that usually spend the same amount of money 
3. Orders amount
4. Usual cart - The customers grouped by the product groups

References
- https://towardsdatascience.com/a-step-by-step-introduction-to-cohort-analysis-in-python-a2cbbd8460ea

### Frequent buyers

In [ ]:

receipt_count = df.groupby('id_card')['id_doc'].count()

plt.figure(figsize=(8, 5))
plt.hist(receipt_count, bins=15)
plt.xlabel('Number of customers')
plt.ylabel('Number of purchases')

In [ ]:
freq_reciepts = (1 / receipt_count[receipt_count > 0] * 30).round()

plt.figure(figsize=(8, 5))
plt.hist(freq_reciepts, bins=30)
plt.xlabel('Number of days between two purchases (on average)')
plt.ylabel('Number of purchases')

Most of people purchase every day. Some of them smth  buy every week

### Average purchase sum

In [ ]:
purchase_sum = df.groupby('id_card')['sum'].sum()
purchase_sum.head()

In [ ]:
plt.figure(figsize=(8, 5))
plt.ylabel('Number of customers')
plt.xlabel('Purchase sum')
plt.hist(purchase_sum, bins=30);

In [ ]:
plt.figure(figsize=(8, 5))
plt.ylabel('Number of customers')
plt.xlabel('Purchase sum')
plt.hist(purchase_sum[purchase_sum < 40000], bins=30);

Most of the customers make purchases for less than 1000 rubles

## Orders amount

In [ ]:
count = df.groupby(['id_card'])['id_doc'].count()

In [ ]:
count.to_frame().hist()

In [ ]:
count[count<100].to_frame().hist()

We can see people that go really frequently and those who visit rare.

### Usual cart

In [ ]:
customers_per_group = df.groupby('id_group')['id_card'].unique().apply(len)

plt.figure(figsize=(8, 5))
plt.hist(customers_per_group.sort_values(ascending=False)[:30], bins=20)
plt.ylabel('Groups count')
plt.ylabel('Number of customers');

In [ ]:
customers_per_group = df.groupby(by=['id_group'])['id_card'].unique().apply(len)
customers_per_group = customers_per_group.sort_values(ascending=False)
customers_per_group[:20].plot(kind='bar')

plt.ylabel('Num of customers');

In [ ]:
sus_groups = p_g[p_g.id_group.isin(customers_per_group[:20].index)]
sus_groups.drop_duplicates('id_group', 'first', inplace=True)
sus_groups['occurence'] = customers_per_group[:20]
sus_groups.sort_values('occurence', ascending=False)
sus_groups[['id_group', 'name_group', 'occurence']].head()

"Кисломолочные продукты" the most popular